# Success Case Analysis

This notebook automates deep analysis of when and why some experiment configurations produce BOTH:
- **Low DrumDeadT30** (e.g., < 200ms) - indicating good sound absorption in the dead zone
- **High DrumLiveT30** (e.g., > 250ms) - indicating good reverberation in the live zone

## Features
- Load cleaned experiment DataFrame from `experiment_metrics.csv`
- Select/adjust thresholds for DrumDeadT30 and DrumLiveT30 interactively
- Filter data for 'success cases' meeting BOTH thresholds
- Compare input/derived parameters between success and other configs
- Visualize key parameter distributions (hist, KDE, boxplot, violin)
- Statistical tests for differences
- Train interpretable classifier (DecisionTree, RandomForest, Logistic Regression)
- Feature importances and SHAP analysis
- Rule extraction: print decision tree rules
- Scatterplots/parallel coordinates to visualize success regions
- Output candidate parameter sets for new experiments

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from typing import Optional, Tuple, List, Dict
import warnings

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# SHAP for model interpretability
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False
    print("SHAP not available. Install with: pip install shap")

# Interactive widgets (optional)
try:
    import ipywidgets as widgets
    from IPython.display import display
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("ipywidgets not available. Install with: pip install ipywidgets")

warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

## 1. Load and Explore Data

In [ ]:
# Load the experiment metrics data
df = pd.read_csv('experiment_metrics.csv')

print(f"Loaded {len(df)} experiments")
print(f"\nColumns: {list(df.columns)}")
print(f"\nData shape: {df.shape}")
df.head()

In [ ]:
# Define feature columns
INPUT_FEATURES = [
    'l_num_reflectors', 'l_reflector_angle', 'l_reflector_depth',
    'l_start_offset', 'l_finish_offset',
    'r_num_reflectors', 'r_reflector_angle', 'r_reflector_depth',
    'r_start_offset', 'r_finish_offset'
]

MAPPED_FEATURES = [
        "LA_angle",
        "SA_angle",
        "LA_num_reflectors",
        "SA_num_reflectors",
        "LA_depth",
        "SA_depth",
        "LA_start_offset",
        "SA_start_offset",
        "LA_finish_offset",
        "SA_finish_offset",
]

DERIVED_FEATURES = [
        #"avg_angle",
        "total_depth",
        "avg_depth",
        "total_reflectors",
        "angle_diff",
        "depth_diff",
        "num_reflectors_diff",
        "start_offset_diff",
        "finish_offset_diff",
]


ALL_FEATURES = MAPPED_FEATURES + DERIVED_FEATURES

# Verify which features exist in the dataframe
available_features = [f for f in ALL_FEATURES if f in df.columns]
print(f"Available features: {len(available_features)} of {len(ALL_FEATURES)}")
print(f"Missing features: {set(ALL_FEATURES) - set(available_features)}")

In [ ]:
# Summary statistics for target metrics
print("Target Metrics Summary:")
print("="*60)
for metric in ['DrumDeadT30', 'DrumLiveT30']:
    if metric in df.columns:
        print(f"\n{metric}:")
        print(f"  Min: {df[metric].min():.2f} ms")
        print(f"  Max: {df[metric].max():.2f} ms")
        print(f"  Mean: {df[metric].mean():.2f} ms")
        print(f"  Median: {df[metric].median():.2f} ms")
        print(f"  Std: {df[metric].std():.2f} ms")

## 2. Define Success Thresholds (Interactive)

In [ ]:
# Default thresholds based on problem statement
DEFAULT_DEAD_T30_THRESHOLD = 140  # ms (max for "success")
DEFAULT_LIVE_T30_THRESHOLD = 250  # ms (min for "success")

# Initialize threshold values
dead_t30_threshold = DEFAULT_DEAD_T30_THRESHOLD
live_t30_threshold = DEFAULT_LIVE_T30_THRESHOLD

def create_success_mask(df: pd.DataFrame, dead_thresh: float, live_thresh: float) -> pd.Series:
    """Create a boolean mask for success cases."""
    return (df['DrumDeadT30'] < dead_thresh) & (df['DrumLiveT30'] > live_thresh)

def count_successes(df: pd.DataFrame, dead_thresh: float, live_thresh: float) -> Tuple[int, int, float]:
    """Count success cases and return statistics."""
    mask = create_success_mask(df, dead_thresh, live_thresh)
    n_success = mask.sum()
    n_total = len(df)
    pct = 100 * n_success / n_total
    return n_success, n_total, pct

# Show initial counts
n_success, n_total, pct = count_successes(df, dead_t30_threshold, live_t30_threshold)
print(f"With thresholds: DrumDeadT30 < {dead_t30_threshold}ms, DrumLiveT30 > {live_t30_threshold}ms")
print(f"Success cases: {n_success} / {n_total} ({pct:.1f}%)")

In [ ]:
# Interactive threshold adjustment (if widgets available)
if WIDGETS_AVAILABLE:
    dead_slider = widgets.FloatSlider(
        value=DEFAULT_DEAD_T30_THRESHOLD,
        min=df['DrumDeadT30'].min(),
        max=df['DrumDeadT30'].max(),
        step=5,
        description='DrumDeadT30 <',
        continuous_update=False
    )
    
    live_slider = widgets.FloatSlider(
        value=DEFAULT_LIVE_T30_THRESHOLD,
        min=df['DrumLiveT30'].min(),
        max=df['DrumLiveT30'].max(),
        step=5,
        description='DrumLiveT30 >',
        continuous_update=False
    )
    
    output = widgets.Output()
    
    def update_thresholds(change):
        global dead_t30_threshold, live_t30_threshold
        dead_t30_threshold = dead_slider.value
        live_t30_threshold = live_slider.value
        with output:
            output.clear_output()
            n_success, n_total, pct = count_successes(df, dead_t30_threshold, live_t30_threshold)
            print(f"Success cases: {n_success} / {n_total} ({pct:.1f}%)")
    
    dead_slider.observe(update_thresholds, names='value')
    live_slider.observe(update_thresholds, names='value')
    
    display(widgets.VBox([dead_slider, live_slider, output]))
    update_thresholds(None)
else:
    print("Interactive widgets not available. Modify thresholds manually below:")
    print(f"dead_t30_threshold = {dead_t30_threshold}")
    print(f"live_t30_threshold = {live_t30_threshold}")

In [ ]:
# Manual threshold adjustment (modify these values as needed)
# Uncomment and modify to override interactive sliders
# dead_t30_threshold = 200
# live_t30_threshold = 250

print(f"\nFinal thresholds: DrumDeadT30 < {dead_t30_threshold}ms, DrumLiveT30 > {live_t30_threshold}ms")
n_success, n_total, pct = count_successes(df, dead_t30_threshold, live_t30_threshold)
print(f"Success cases: {n_success} / {n_total} ({pct:.1f}%)")

## 3. Filter Success Cases

In [ ]:
# Create success label
df['is_success'] = create_success_mask(df, dead_t30_threshold, live_t30_threshold)

# Split data
df_success = df[df['is_success']].copy()
df_other = df[~df['is_success']].copy()

print(f"Success cases: {len(df_success)}")
print(f"Other cases: {len(df_other)}")

# Display success cases
print("\n" + "="*60)
print("SUCCESS CASES:")
print("="*60)
success_display = df_success[['experiment', 'DrumDeadT30', 'DrumLiveT30'] + available_features[:5]].head(20)
display(success_display)

## 4. Compare Parameters: Success vs Other Configurations

In [ ]:
def compare_groups(df_success: pd.DataFrame, df_other: pd.DataFrame, 
                   features: List[str]) -> pd.DataFrame:
    """Compare feature statistics between success and other groups."""
    comparison = []
    
    for feature in features:
        if feature not in df_success.columns:
            continue
            
        success_vals = df_success[feature].dropna()
        other_vals = df_other[feature].dropna()
        
        if len(success_vals) < 2 or len(other_vals) < 2:
            continue
        
        # Statistical test (Mann-Whitney U for non-parametric comparison)
        stat, p_value = stats.mannwhitneyu(success_vals, other_vals, alternative='two-sided')
        
        # Effect size (Cohen's d approximation)
        pooled_std = np.sqrt((success_vals.std()**2 + other_vals.std()**2) / 2)
        if pooled_std > 0:
            cohens_d = (success_vals.mean() - other_vals.mean()) / pooled_std
        else:
            cohens_d = 0
        
        comparison.append({
            'feature': feature,
            'success_mean': success_vals.mean(),
            'success_std': success_vals.std(),
            'other_mean': other_vals.mean(),
            'other_std': other_vals.std(),
            'diff_mean': success_vals.mean() - other_vals.mean(),
            'cohens_d': cohens_d,
            'p_value': p_value,
            'significant': p_value < 0.05
        })
    
    return pd.DataFrame(comparison).sort_values('p_value')

comparison_df = compare_groups(df_success, df_other, available_features)
print("Parameter Comparison: Success vs Other Configurations")
print("="*80)
display(comparison_df.round(4))

In [ ]:
# Highlight significant differences
significant_features = comparison_df[comparison_df['significant']]['feature'].tolist()
print(f"\nFeatures with significant differences (p < 0.05): {len(significant_features)}")
for feat in significant_features:
    row = comparison_df[comparison_df['feature'] == feat].iloc[0]
    direction = "higher" if row['diff_mean'] > 0 else "lower"
    print(f"  - {feat}: Success cases are {direction} (diff={row['diff_mean']:.3f}, p={row['p_value']:.4f})")

## 5. Visualize Parameter Distributions

In [ ]:
def plot_distribution_comparison(df: pd.DataFrame, feature: str, 
                                  figsize: tuple = (14, 4)) -> plt.Figure:
    """Plot histograms, KDE, boxplot, and violin plots for a feature."""
    fig, axes = plt.subplots(1, 4, figsize=figsize)
    
    success_vals = df[df['is_success']][feature].dropna()
    other_vals = df[~df['is_success']][feature].dropna()
    
    # Histogram
    axes[0].hist(other_vals, bins=20, alpha=0.6, label='Other', color='gray')
    axes[0].hist(success_vals, bins=20, alpha=0.8, label='Success', color='green')
    axes[0].set_xlabel(feature)
    axes[0].set_ylabel('Count')
    axes[0].set_title('Histogram')
    axes[0].legend()
    
    # KDE
    if len(success_vals) > 1 and len(other_vals) > 1:
        sns.kdeplot(other_vals, ax=axes[1], label='Other', color='gray', fill=True, alpha=0.3)
        sns.kdeplot(success_vals, ax=axes[1], label='Success', color='green', fill=True, alpha=0.5)
    axes[1].set_xlabel(feature)
    axes[1].set_title('KDE')
    axes[1].legend()
    
    # Boxplot
    box_data = [other_vals, success_vals]
    bp = axes[2].boxplot(box_data, labels=['Other', 'Success'], patch_artist=True)
    bp['boxes'][0].set_facecolor('gray')
    bp['boxes'][1].set_facecolor('green')
    axes[2].set_ylabel(feature)
    axes[2].set_title('Boxplot')
    
    # Violin plot
    sns.violinplot(x='is_success', y=feature, data=df, ax=axes[3], 
                   hue='is_success', palette={True: 'green', False: 'gray'}, legend=False)
    axes[3].set_xticklabels(['Other', 'Success'])
    axes[3].set_xlabel('')
    axes[3].set_title('Violin Plot')
    
    plt.suptitle(f'Distribution Comparison: {feature}', fontsize=12, fontweight='bold')
    plt.tight_layout()
    return fig

# Plot distributions for top significant features
top_features = comparison_df.head(6)['feature'].tolist()
for feature in top_features:
    plot_distribution_comparison(df, feature)
    plt.show()

In [ ]:
# Summary violin plot for all features
n_features = len(available_features)
n_cols = 4
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4*n_rows))
axes = axes.flatten()

for idx, feature in enumerate(available_features):
    if feature in df.columns:
        sns.violinplot(x='is_success', y=feature, data=df, ax=axes[idx],
                      hue='is_success', palette={True: 'green', False: 'gray'}, legend=False)
        axes[idx].set_xticklabels(['Other', 'Success'])
        axes[idx].set_xlabel('')
        axes[idx].set_title(feature, fontsize=10)

# Hide unused axes
for idx in range(len(available_features), len(axes)):
    axes[idx].set_visible(False)

plt.suptitle('All Features: Success vs Other Distributions', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 6. Statistical Tests for Differences

In [ ]:
def run_statistical_tests(df_success: pd.DataFrame, df_other: pd.DataFrame,
                          features: List[str]) -> pd.DataFrame:
    """Run multiple statistical tests comparing groups."""
    results = []
    
    for feature in features:
        if feature not in df_success.columns:
            continue
            
        success_vals = df_success[feature].dropna()
        other_vals = df_other[feature].dropna()
        
        if len(success_vals) < 3 or len(other_vals) < 3:
            continue
        
        # Mann-Whitney U test (non-parametric)
        mw_stat, mw_p = stats.mannwhitneyu(success_vals, other_vals, alternative='two-sided')
        
        # T-test (parametric)
        t_stat, t_p = stats.ttest_ind(success_vals, other_vals)
        
        # Kolmogorov-Smirnov test (distribution difference)
        ks_stat, ks_p = stats.ks_2samp(success_vals, other_vals)
        
        results.append({
            'feature': feature,
            'mw_stat': mw_stat,
            'mw_p_value': mw_p,
            't_stat': t_stat,
            't_p_value': t_p,
            'ks_stat': ks_stat,
            'ks_p_value': ks_p,
            'any_significant': (mw_p < 0.05) or (t_p < 0.05) or (ks_p < 0.05)
        })
    
    return pd.DataFrame(results).sort_values('mw_p_value')

stat_tests = run_statistical_tests(df_success, df_other, available_features)
print("Statistical Tests: Success vs Other")
print("="*100)
print("MW = Mann-Whitney U, T = T-test, KS = Kolmogorov-Smirnov")
print()
display(stat_tests.round(4))

## 7. Train Interpretable Classifiers

In [ ]:
# Prepare data for classification
X = df[available_features].copy()
y = df['is_success'].astype(int)

# Handle missing values
X = X.fillna(X.mean())

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set: {len(X_train)} samples ({y_train.sum()} success)")
print(f"Test set: {len(X_test)} samples ({y_test.sum()} success)")

In [ ]:
# Scale features for Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train classifiers
classifiers = {
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

results = {}
for name, clf in classifiers.items():
    if name == 'Logistic Regression':
        clf.fit(X_train_scaled, y_train)
        y_pred = clf.predict(X_test_scaled)
        cv_scores = cross_val_score(clf, X_train_scaled, y_train, cv=5)
    else:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        cv_scores = cross_val_score(clf, X_train, y_train, cv=5)
    
    results[name] = {
        'model': clf,
        'accuracy': accuracy_score(y_test, y_pred),
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'y_pred': y_pred
    }
    
    print(f"\n{name}:")
    print(f"  Test Accuracy: {results[name]['accuracy']:.4f}")
    print(f"  CV Accuracy: {results[name]['cv_mean']:.4f} (+/- {results[name]['cv_std']:.4f})")

In [ ]:
# Classification reports
for name, result in results.items():
    print(f"\n{'='*60}")
    print(f"{name} - Classification Report")
    print("="*60)
    print(classification_report(y_test, result['y_pred'], target_names=['Other', 'Success']))

## 8. Feature Importances

In [ ]:
def plot_feature_importances(importance_dict: Dict[str, float], title: str,
                              figsize: tuple = (10, 6)) -> plt.Figure:
    """Plot feature importances as horizontal bar chart."""
    sorted_items = sorted(importance_dict.items(), key=lambda x: abs(x[1]), reverse=True)
    features = [item[0] for item in sorted_items]
    importances = [item[1] for item in sorted_items]
    
    fig, ax = plt.subplots(figsize=figsize)
    colors = ['green' if imp > 0 else 'red' for imp in importances]
    
    y_pos = range(len(features))
    ax.barh(y_pos, importances, color=colors, alpha=0.7)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(features)
    ax.set_xlabel('Importance')
    ax.set_title(title)
    ax.axvline(x=0, color='black', linewidth=0.5)
    
    plt.tight_layout()
    return fig

# Decision Tree feature importances
dt_clf = results['Decision Tree']['model']
dt_importances = dict(zip(available_features, dt_clf.feature_importances_))
plot_feature_importances(dt_importances, 'Decision Tree Feature Importances')
plt.show()

# Random Forest feature importances
rf_clf = results['Random Forest']['model']
rf_importances = dict(zip(available_features, rf_clf.feature_importances_))
plot_feature_importances(rf_importances, 'Random Forest Feature Importances')
plt.show()

# Logistic Regression coefficients (scaled)
lr_clf = results['Logistic Regression']['model']
lr_importances = dict(zip(available_features, lr_clf.coef_[0]))
plot_feature_importances(lr_importances, 'Logistic Regression Coefficients (standardized)')
plt.show()

In [ ]:
# Combined feature importance ranking
importance_df = pd.DataFrame({
    'feature': available_features,
    'dt_importance': [dt_importances[f] for f in available_features],
    'rf_importance': [rf_importances[f] for f in available_features],
    'lr_coef': [abs(lr_importances[f]) for f in available_features]
})

# Normalize and compute average rank
for col in ['dt_importance', 'rf_importance', 'lr_coef']:
    importance_df[f'{col}_rank'] = importance_df[col].rank(ascending=False)

importance_df['avg_rank'] = importance_df[['dt_importance_rank', 'rf_importance_rank', 'lr_coef_rank']].mean(axis=1)
importance_df = importance_df.sort_values('avg_rank')

print("\nCombined Feature Importance Ranking:")
print("="*80)
display(importance_df[['feature', 'dt_importance', 'rf_importance', 'lr_coef', 'avg_rank']].round(4))

## 9. SHAP Analysis

In [ ]:
if SHAP_AVAILABLE:
    # SHAP for Random Forest
    print("Computing SHAP values for Random Forest...")
    explainer_rf = shap.TreeExplainer(rf_clf)
    shap_values_rf = explainer_rf.shap_values(X_test)
    
    # Handle different SHAP versions - newer versions return different format
    # For binary classification, get SHAP values for positive class (success)
    if isinstance(shap_values_rf, list):
        # Older SHAP: returns list of arrays [class_0_values, class_1_values]
        shap_values_success = shap_values_rf[1]
    else:
        # Newer SHAP: returns single array or Explanation object
        shap_values_success = shap_values_rf
    
    # SHAP summary plot for success class
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values_success, X_test, feature_names=available_features, show=False)
    plt.title('SHAP Summary Plot (Random Forest) - Success Class')
    plt.tight_layout()
    plt.show()
    
    # SHAP bar plot
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_values_success, X_test, feature_names=available_features, 
                      plot_type='bar', show=False)
    plt.title('SHAP Feature Importance (Random Forest) - Success Class')
    plt.tight_layout()
    plt.show()
else:
    print("SHAP analysis skipped (shap package not available)")
    print("Install with: pip install shap")

In [ ]:
if SHAP_AVAILABLE:
    # SHAP dependence plots for top features
    top_rf_features = sorted(rf_importances.items(), key=lambda x: x[1], reverse=True)[:4]
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    # Convert shap_values to numpy array if it's an Explanation object
    if hasattr(shap_values_success, 'values'):
        shap_values_array = shap_values_success.values
    else:
        shap_values_array = np.array(shap_values_success)
    
    # Convert X_test to numpy array if it's a DataFrame
    X_test_array = X_test.values if hasattr(X_test, 'values') else np.array(X_test)
    
    for idx, (feature, _) in enumerate(top_rf_features):
        feature_idx = available_features.index(feature)
        # Use interaction_index=None to avoid the automatic interaction detection that causes errors
        shap.dependence_plot(feature_idx, shap_values_array, X_test_array, 
                            feature_names=available_features, ax=axes[idx], show=False,
                            interaction_index=None)
        axes[idx].set_title(f'SHAP Dependence: {feature}')
    
    plt.suptitle('SHAP Dependence Plots for Top Features', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 10. Rule Extraction from Decision Tree

In [ ]:
# Visualize Decision Tree
plt.figure(figsize=(20, 12))
plot_tree(dt_clf, feature_names=available_features, class_names=['Other', 'Success'],
          filled=True, rounded=True, fontsize=8)
plt.title('Decision Tree for Success Classification', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Print text rules
print("\n" + "="*80)
print("DECISION TREE RULES FOR SUCCESS")
print("="*80)
tree_rules = export_text(dt_clf, feature_names=available_features)
print(tree_rules)

In [ ]:
def extract_success_paths(tree, feature_names: List[str]) -> List[str]:
    """Extract decision paths that lead to 'Success' classification."""
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != -2 else "undefined!"
        for i in tree_.feature
    ]
    
    paths = []
    
    def recurse(node, path):
        if tree_.feature[node] != -2:  # Not a leaf
            name = feature_name[node]
            threshold = tree_.threshold[node]
            
            # Left branch (<=)
            recurse(tree_.children_left[node], path + [f"{name} <= {threshold:.2f}"])
            # Right branch (>)
            recurse(tree_.children_right[node], path + [f"{name} > {threshold:.2f}"])
        else:  # Leaf node
            # Check if this is a "success" leaf (class 1 has more samples)
            values = tree_.value[node][0]
            if values[1] > values[0]:  # More successes than others
                confidence = values[1] / sum(values)
                paths.append({
                    'rules': path,
                    'confidence': confidence,
                    'n_success': int(values[1]),
                    'n_other': int(values[0])
                })
    
    recurse(0, [])
    return paths

success_paths = extract_success_paths(dt_clf, available_features)

print("\n" + "="*80)
print("RULES LEADING TO SUCCESS (High confidence paths)")
print("="*80)
for i, path in enumerate(sorted(success_paths, key=lambda x: x['confidence'], reverse=True)):
    print(f"\nPath {i+1} (Confidence: {path['confidence']:.1%}, Samples: {path['n_success']} success, {path['n_other']} other):")
    for rule in path['rules']:
        print(f"  - {rule}")

## 11. Visualize Success Regions

In [ ]:
# Scatter plot of DrumDeadT30 vs DrumLiveT30 with success regions
plt.figure(figsize=(10, 8))

plt.scatter(df[~df['is_success']]['DrumDeadT30'], 
            df[~df['is_success']]['DrumLiveT30'],
            c='gray', alpha=0.5, label='Other', s=50)
plt.scatter(df[df['is_success']]['DrumDeadT30'], 
            df[df['is_success']]['DrumLiveT30'],
            c='green', alpha=0.8, label='Success', s=80, edgecolors='black')

# Draw threshold lines
plt.axvline(x=dead_t30_threshold, color='red', linestyle='--', 
            label=f'DrumDeadT30 threshold ({dead_t30_threshold}ms)')
plt.axhline(y=live_t30_threshold, color='blue', linestyle='--',
            label=f'DrumLiveT30 threshold ({live_t30_threshold}ms)')

# Shade success region
plt.fill_between([df['DrumDeadT30'].min(), dead_t30_threshold],
                 live_t30_threshold, df['DrumLiveT30'].max(),
                 alpha=0.1, color='green', label='Success Region')

plt.xlabel('DrumDeadT30 (ms)')
plt.ylabel('DrumLiveT30 (ms)')
plt.title('Success Region: Low DrumDeadT30 AND High DrumLiveT30')
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots for top important features vs targets
top_features_for_scatter = importance_df.head(4)['feature'].tolist()

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, feature in enumerate(top_features_for_scatter):
    ax = axes[idx]
    
    # Create scatter with color by success
    scatter = ax.scatter(df[feature], df['DrumDeadT30'], 
                        c=df['is_success'].map({True: 'green', False: 'gray'}),
                        alpha=0.6, s=50)
    
    ax.axhline(y=dead_t30_threshold, color='red', linestyle='--', alpha=0.7)
    ax.set_xlabel(feature)
    ax.set_ylabel('DrumDeadT30 (ms)')
    ax.set_title(f'{feature} vs DrumDeadT30')

plt.suptitle('Top Features vs DrumDeadT30 (Green = Success)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Parallel coordinates plot
from pandas.plotting import parallel_coordinates

# Select subset of features for readability
parallel_features = importance_df.head(6)['feature'].tolist()

# Prepare data for parallel coordinates
parallel_df = df[parallel_features + ['is_success']].copy()

# Normalize features for better visualization
for col in parallel_features:
    parallel_df[col] = (parallel_df[col] - parallel_df[col].min()) / (parallel_df[col].max() - parallel_df[col].min())

parallel_df['Group'] = parallel_df['is_success'].map({True: 'Success', False: 'Other'})

plt.figure(figsize=(14, 6))
parallel_coordinates(parallel_df.drop('is_success', axis=1), 'Group', 
                     color=['gray', 'green'], alpha=0.5)
plt.title('Parallel Coordinates: Success vs Other Configurations')
plt.ylabel('Normalized Value')
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# Pairplot for top features
pairplot_features = importance_df.head(4)['feature'].tolist()
pairplot_df = df[pairplot_features + ['is_success']].copy()

g = sns.pairplot(pairplot_df, hue='is_success', 
                 palette={True: 'green', False: 'gray'},
                 diag_kind='kde', plot_kws={'alpha': 0.5})
g.fig.suptitle('Pairplot: Top Features by Success', y=1.02)
plt.show()

## 12. Output Candidate Parameter Sets for New Experiments

In [ ]:
def generate_candidate_parameters(df_success: pd.DataFrame, 
                                   features: List[str],
                                   n_candidates: int = 5,
                                   strategy: str = 'mean') -> pd.DataFrame:
    """Generate candidate parameter sets based on success cases.
    
    Args:
        df_success: DataFrame of success cases
        features: List of feature columns
        n_candidates: Number of candidate sets to generate
        strategy: 'mean', 'median', 'random', or 'optimized'
    
    Returns:
        DataFrame with candidate parameter sets
    """
    candidates = []
    
    if strategy == 'mean':
        # Use mean values from success cases
        candidate = {f: df_success[f].mean() for f in features if f in df_success.columns}
        candidate['strategy'] = 'success_mean'
        candidates.append(candidate)
        
    elif strategy == 'median':
        # Use median values from success cases
        candidate = {f: df_success[f].median() for f in features if f in df_success.columns}
        candidate['strategy'] = 'success_median'
        candidates.append(candidate)
        
    elif strategy == 'random':
        # Random samples from success cases
        for i in range(min(n_candidates, len(df_success))):
            sample = df_success.sample(1).iloc[0]
            candidate = {f: sample[f] for f in features if f in df_success.columns}
            candidate['strategy'] = f'random_sample_{i+1}'
            candidates.append(candidate)
            
    elif strategy == 'optimized':
        # Generate candidates around the best success cases
        # Sort by combined objective (low dead, high live)
        df_sorted = df_success.copy()
        df_sorted['score'] = -df_sorted['DrumDeadT30'] + df_sorted['DrumLiveT30']
        df_sorted = df_sorted.sort_values('score', ascending=False)
        
        for i in range(min(n_candidates, len(df_sorted))):
            sample = df_sorted.iloc[i]
            candidate = {f: sample[f] for f in features if f in df_sorted.columns}
            candidate['strategy'] = f'top_performer_{i+1}'
            candidate['DrumDeadT30'] = sample['DrumDeadT30']
            candidate['DrumLiveT30'] = sample['DrumLiveT30']
            candidates.append(candidate)
    
    return pd.DataFrame(candidates)

# Generate candidates using different strategies
print("\n" + "="*80)
print("CANDIDATE PARAMETER SETS FOR NEW EXPERIMENTS")
print("="*80)

# Mean-based candidate
candidates_mean = generate_candidate_parameters(df_success, INPUT_FEATURES, strategy='mean')
print("\n1. Based on SUCCESS CASE MEANS:")
display(candidates_mean.round(2))

# Median-based candidate
candidates_median = generate_candidate_parameters(df_success, INPUT_FEATURES, strategy='median')
print("\n2. Based on SUCCESS CASE MEDIANS:")
display(candidates_median.round(2))

# Top performers
candidates_top = generate_candidate_parameters(df_success, INPUT_FEATURES, n_candidates=5, strategy='optimized')
print("\n3. TOP PERFORMING SUCCESS CASES:")
display(candidates_top.round(2))

In [ ]:
# Recommend parameter ranges based on success cases
print("\n" + "="*80)
print("RECOMMENDED PARAMETER RANGES (from success cases)")
print("="*80)

for feature in INPUT_FEATURES:
    if feature in df_success.columns:
        vals = df_success[feature]
        print(f"\n{feature}:")
        print(f"  Range: [{vals.min():.2f}, {vals.max():.2f}]")
        print(f"  Mean: {vals.mean():.2f} (±{vals.std():.2f})")
        print(f"  Median: {vals.median():.2f}")
        print(f"  IQR: [{vals.quantile(0.25):.2f}, {vals.quantile(0.75):.2f}]")

In [ ]:
# Export candidate parameters to CSV
all_candidates = pd.concat([
    candidates_mean,
    candidates_median,
    candidates_top
], ignore_index=True)

# Save to CSV
output_path = 'candidate_parameters.csv'
all_candidates.to_csv(output_path, index=False)
print(f"\nCandidate parameters saved to: {output_path}")
display(all_candidates.round(2))

## 13. Summary and Conclusions

In [ ]:
print("\n" + "="*80)
print("ANALYSIS SUMMARY")
print("="*80)

print(f"\n1. DATA OVERVIEW:")
print(f"   - Total experiments: {len(df)}")
print(f"   - Success cases (DrumDeadT30 < {dead_t30_threshold}ms AND DrumLiveT30 > {live_t30_threshold}ms): {len(df_success)} ({100*len(df_success)/len(df):.1f}%)")

print(f"\n2. KEY DIFFERENTIATING FEATURES (statistically significant):")
for feat in significant_features[:5]:
    row = comparison_df[comparison_df['feature'] == feat].iloc[0]
    direction = "↑" if row['diff_mean'] > 0 else "↓"
    print(f"   - {feat}: Success {direction} (diff={row['diff_mean']:.3f}, p={row['p_value']:.4f})")

print(f"\n3. CLASSIFIER PERFORMANCE:")
for name, result in results.items():
    print(f"   - {name}: {result['accuracy']:.1%} test accuracy, {result['cv_mean']:.1%} CV accuracy")

print(f"\n4. TOP FEATURES BY IMPORTANCE (Random Forest):")
for feat, imp in sorted(rf_importances.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"   - {feat}: {imp:.4f}")

print(f"\n5. DECISION RULES FOR SUCCESS:")
if success_paths:
    top_path = sorted(success_paths, key=lambda x: x['confidence'], reverse=True)[0]
    print(f"   Best rule (confidence: {top_path['confidence']:.1%}):")
    for rule in top_path['rules']:
        print(f"     - {rule}")

## Next Steps

This notebook provides a foundation for success case analysis. Future expansions could include:

1. **Additional FOM combinations**: Extend analysis to other metric pairs (e.g., VocalT30, DrumDiffusion)
2. **Multi-objective optimization**: Find Pareto-optimal configurations
3. **Sensitivity analysis**: How robust are success cases to parameter variations?
4. **Bayesian optimization**: Suggest optimal next experiments to run
5. **Ensemble methods**: Combine multiple models for more robust predictions